In [ ]:
import requests
import json
import re

In [ ]:
header = {'Authorization': 'Bearer 000000-test'} #Your auth goes here, see https://developer.yammer.com/docs/authentication-1

In [ ]:
all_users = []
for i in range(1,31):  #modify range depending on how many users your organisation has 
    r = requests.get(r'https://www.yammer.com/api/v1/users.json?page={}'.format(i), headers=header )
    e = json.loads(r.text)
    all_users.extend(e)

In [ ]:
with open('allusers.json', 'w') as outfile:
    json.dump(all_users, outfile)

In [ ]:
#This produces a very rough measure of user activity.  

#As far as I can tell, there's no easy way of asking
#the api how many messages/interactions the user has 
#has in the past x days


import re
from dateutil.parser import parse
import datetime
from time import sleep
activity_counts = []
for count, user in enumerate(all_users):
    id = str(user["id"])
    r = requests.get(r'https://www.yammer.com/api/v1/messages/from_user/{}.json?limit=10000&threaded=extended&exclude_own_messages_from_unseen=false'.format(id), headers=header )
    
    found = re.findall(r'"sender_id"\:' + id +  '.{1,100}"network_id"', r.text)
    found = [json.loads("{" + i + ': "1" }') for i in found]
    today = datetime.datetime.today().date()
    found = [parse(i["created_at"]).date() for i in found]
    found_six_months = [i for i in found if (today - i) < datetime.timedelta(days=180)]
    
    foundall = re.findall(r'"sender_id"\:' + str(id), r.text)

    activity_counts.append({"id":id, "full_name": user["full_name"], "messages_in_last_180_days": len(found_six_months), "all_messages":len(foundall) })
    print (count,{"id":id, "full_name": user["full_name"], "messages_in_last_180_days": len(found_six_months), "all_messages": len(foundall)})
              
    
        
    sleep(1) #To prevent being banned for exceeding the rate limit

with open('allactivity.json', 'w') as outfile:
        json.dump(activity_counts, outfile)

In [ ]:
#Get data on line managers and staff
all_relationships = []

from time import sleep
for count, i in enumerate(all_users):
    id = i["id"]
    r = requests.get(r'https://www.yammer.com/api/v1/relationships.json?user_id={}'.format(id), headers=header )
    e = json.loads(r.text)
    all_relationships.append(e)
    sleep(1)
    print(count)
        
with open('allrelationships.json', 'w') as outfile:
    json.dump(all_relationships, outfile)